In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')


In [26]:
trainingData = pd.read_csv('./data/train.csv')
testData = pd.read_csv('./data/test.csv')


In [27]:
from collections import Counter
import re

# trainingDataのコピーを作成 コピーした奴は、中身をいじる
trainingDataCopy = trainingData.copy()


def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""


# 敬称を新たな列として追加
trainingDataCopy['Title'] = trainingDataCopy['Name'].apply(get_title)
# trainingDataCopy_test['Title'] = trainingDataCopy_test['Name'].apply(get_title)

titles = list(trainingDataCopy['Title'])
# title_namesのリストの中にある敬称を持つ人の平均年齢を算出し、dictに格納　
title_ages = {}
for title in titles:
    title_ages[title] = trainingDataCopy[trainingDataCopy['Title']
                                         == title]['Age'].mean()

# リスト2つからdictを作成

# 年齢が欠損しているレコードを、その人の敬称に対応する平均年齢で補完
trainingDataCopy.loc[trainingDataCopy['Age'].isnull(), 'Age'] = trainingDataCopy[trainingData['Age'].isnull()].apply(
    lambda row: title_ages[row['Title']], axis=1)
# trainingDataCopy_test.loc[trainingDataCopy_test['Age'].isnull(), 'Age'] = trainingDataCopy_test[trainingDataCopy_test['Age'].isnull()].apply(
#     lambda row: title_ages[row['Title']], axis=1)


In [28]:
trainingDataCopy.drop('Cabin', axis=1, inplace=True)
trainingDataCopy['Embarked'].fillna('S', inplace=True)

trainingDataCopy.isnull().sum()


PassengerId    0
Perished       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Title          0
dtype: int64

In [29]:
trainingDataCopy.replace({'Sex': {'male': 0, 'female': 1}}, inplace=True)

embarked_ohe = trainingDataCopy['Embarked']

embarked_ohe = pd.get_dummies(embarked_ohe)

trainingDataCopy = pd.concat([trainingDataCopy, embarked_ohe], axis=1)

trainingDataCopy.drop('Embarked', axis=1, inplace=True)

trainingDataCopy.head()


,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Title,C,Q,S
0,1,1,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,Mr,False,False,True
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,Mrs,True,False,False
2,3,0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,Miss,False,False,True
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,Mrs,False,False,True
4,5,1,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,Mr,False,False,True


In [31]:
X = trainingDataCopy.iloc[:, 2:].values
y = trainingDataCopy.iloc[:, 1].values


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


param_grid = {'max_depth': [3, 5, 6, 7, 8, 9],
              'min_samples_leaf': [1, 2, 3, 4, 5]}


param_grid = {'max_depth': [3, 5, 7],
              'min_samples_leaf': [1, 2, 4]}


X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=42)


rfc_gs = GridSearchCV(RandomForestClassifier(
    n_estimators=100, n_jobs=-1, random_state=42), param_grid, cv=5)
rfc_gs.fit(X, y)

print('Best Parameters: {}'.format(rfc_gs.best_params_))
print('CV Score: {}'.format(round(rfc_gs.best_score_, 3)))


ValueError: 
All the 45 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\kouda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\kouda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py", line 345, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kouda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kouda\AppData\Local\Progra

In [ ]:
param_grid = {'max_depth': [3, 5, 7],
              'min_samples_leaf': [1, 2, 4]}
